In [1]:
# ARIS_DIR = '/home/petey/code/fish_eye/kenai-txt/'
MM_DIR = '/Users/sstat/Projects/Fish_Counting/candidate_elwha_batch4_data/'

import json
import os
from pathlib import Path
import re
import tqdm

# River direction is omitted in Elwha manual markings but I think they're all 'Left To Right'

In [2]:
RE1 = re.compile( r'Total Fish\s+=[ ]+([0-9]+)\n'
                + r'Upstream\s+=[ ]+([0-9]+)\n'
                + r'Downstream\s+=[ ]+([0-9]+)\n'
                + r'\?\?\s+=[ ]+([0-9]+)\n'
                + r'\n'
                + r'Total Frames\s+=[ ]+([0-9]+)\n'
                + r'Expected Frames\s+=[ ]+([0-9]+)\n'
                + r'Total Time\s+=[ ]+([0-9]{2}:[0-9]{2}:[0-9]{2})\n'
                + r'Expected Time\s+=[ ]+([0-9]{2}:[0-9]{2}:[0-9]{2})\n'
                + r'\n'
                + r'Upstream Motion\s+=[ ]+(.*)\n'
                + r'\n'
                + r'Count\s+File\s+Name:[ ]+(?:.*)\n'
                + r'Editor ID\s+=[ ]+(\w*)\n'
                + r'Intensity\s+=[ ]+(.*)\n'
                + r'Threshold\s+=[ ]+(.*)\n'
                + r'Window Start\s+=[ ]+(.*)\n'
                + r'Window End\s+=[ ]+(.*)\n'
                + r'Water Temperature\s+=[ ]+(.*)degC\n'
                + r'\n'
                + r'\n'
                + r'(?:.*)\n+(?:.*)\n\-+'
                + r'\n([\w\s\S]*?)\n\n+')

RE2 = re.compile(r'(.*?)\s+(.*?)\s+(.*?)\s+(.*?)\s+(.*?)\s+(.*?)\s+(.*?)\s+(.*?)\s+(.*?)\s+(.*?)\s+(.*?)\s+(.*?)\s+'
               + r'((?:.*?)\s+(?:.*?)\s+(?:.*?)\s+(?:.*?)\s+(?:.*?))\s+'
               + r'((?:.*?)\s+(?:.*?)\s+(?:.*?)\s+(?:.*?)\s+(?:.*?))\s+')

In [3]:
unmatched = []
json_data = {}

for txt in tqdm.tqdm(list(Path(MM_DIR).rglob('*.txt'))):
    txt = str(txt)
    
    with open(txt) as file:
        contents = file.read()

    matched = RE1.search(contents)

    if not matched:
        unmatched.append(txt)
        continue
    
    data = matched.groups()

    info = {}
    info['tot_fish'] = int(data[0])
    info['num_up'] = int(data[1])
    info['num_down'] = int(data[2])
    info['num_unknown'] = int(data[3])
    info['tot_frames'] = int(data[4])
    info['exp_frames'] = int(data[5])
    info['tot_time'] = data[6]
    info['exp_time'] = data[7]
    info['upstream_motion'] = data[8]
    info['editor_id'] = data[9]
    info['intensity'] = data[10]
    info['threshold'] = data[11]
    info['window_start'] = float(data[12])
    info['window_end'] = float(data[13])
    info['water_temp'] = data[14]
    entry = {'info': info}

    frames = {}
    entry['annotations'] = frames
    json_data[txt] = entry
    
    annotations = re.split(r'\n', data[15])
    if annotations == ['']:
        continue

    for i in range(len(annotations)):
        annotations[i] = annotations[i].strip()
        matched = RE2.match(annotations[i])
        if not matched:
            continue
        data = matched.groups()
        frame = {}
        frame['id'] = int(data[1])
        frame['frame_num'] = int(data[2])
        frame['dir'] = data[3]
        frame['R'] = float(data[4])
        frame['theta'] = float(data[5])
        frame['L'] = float(data[6])
        frame['dR'] = float(data[7])
        frame['L_dR'] = float(data[8])
        frame['aspect'] = float(data[9])
        frame['time'] = data[10]
        frame['date'] = data[11]
        frame['latitude'] = data[12]
        frame['longitude'] = data[13]
        frames[frame['id']] = frame
        
print(len(json_data), len(unmatched))
display(unmatched)

100%|██████████| 287/287 [00:00<00:00, 3187.64it/s]

287 0


[]

In [4]:
unmatched

[]

In [5]:
json_data


{'/Users/sstat/Projects/Fish_Counting/candidate_elwha_batch4_data/FCe_2018-07-17_210000_ID_.txt': {'info': {'tot_fish': 1,
   'num_up': 1,
   'num_down': 0,
   'num_unknown': 0,
   'tot_frames': 12193,
   'exp_frames': 13502,
   'tot_time': '00:27:04',
   'exp_time': '00:29:58',
   'upstream_motion': 'Undefined',
   'editor_id': '',
   'intensity': '80.0 dB',
   'threshold': '0.0 dB',
   'window_start': 0.68,
   'window_end': 15.31,
   'water_temp': '17 '},
  'annotations': {1: {'id': 1,
    'frame_num': 6834,
    'dir': 'Up',
    'R': 13.01,
    'theta': 99.0,
    'L': 0.0,
    'dR': 0.0,
    'L_dR': 0.0,
    'aspect': 0.0,
    'time': '21:16:54',
    'date': '2018-07-17',
    'latitude': 'N 00 d  0.000 m',
    'longitude': 'E 000 d  0.000 m'}}},
 '/Users/sstat/Projects/Fish_Counting/candidate_elwha_batch4_data/FCe_2018-07-13_000000_ID_.txt': {'info': {'tot_fish': 10,
   'num_up': 10,
   'num_down': 0,
   'num_unknown': 0,
   'tot_frames': 12749,
   'exp_frames': 13502,
   'tot_time':